In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import spacy
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

data = pd.read_csv('/kaggle/input/bankOffers/Transaction_Naration_Data_Set.csv')  # Replace 'transactions.csv' with the actual filename

# Perform any necessary preprocessing steps, such as removing noise or irrelevant characters


In [ ]:
!python -m spacy download en_core_web_md 

In [20]:
from nltk.corpus import words

# Load the English word corpus
nltk.download('words')
english_words = set(words.words())

# Function to check if a word is an English word
def is_english_word(word):
    return word.lower() in english_words

# Filter rows that contain English words in the 'Naration' column
data = data[data['NARATION'].apply(lambda x: isinstance(x, str) and any(is_english_word(word) for word in x.split()))]

[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [22]:
data['NARATION'] = data['NARATION'].str.replace('[^A-Za-z0-9]+', ' ', regex=True)
data.head()

/tmp/ipykernel_34/3125027333.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['NARATION'] = data['NARATION'].str.replace('[^A-Za-z0-9]+', ' ', regex=True)


,Unnamed: 0,AC_ID,AMOUNT,DOC_DATE,NARATION,DR_CR
0,0,3644877,342800.0,01-NOV-22,Cash Withdraw,Debit
1,1,1175256,342800.0,01-NOV-22,Cash Withdrawal From A C No,Credit
2,2,2298226,1500.0,01-NOV-22,Cash withdraw from Micro Merchant point MM A C,Debit
3,3,4066427,1500.0,01-NOV-22,Credit Against Merchant Cash Withdraw Service ...,Credit
4,4,1978564,2550.0,01-NOV-22,Cash withdraw from Micro Merchant point MM A C,Debit


In [ ]:
# Load the pre-trained SpaCy model
nlp = spacy.load("en_core_web_sm")

def eraseName(text):
    doc = nlp(text)
    # Remove named entities of type "PERSON" and "GPE"
    cleaned_text = []
    for token in doc:
        if token.ent_type_ not in ["PERSON", "GPE"]:
            cleaned_text.append(token.text)

    # Join the cleaned tokens back into a string
    cleaned_text = " ".join(cleaned_text)
    return cleaned_text

data['NARATION'] = data['NARATION'].apply(eraseName)
print(data.head)

In [25]:
data.head()

,Unnamed: 0,AC_ID,AMOUNT,DOC_DATE,NARATION,DR_CR
0,0,3644877,342800.0,01-NOV-22,Cash Withdraw,Debit
1,1,1175256,342800.0,01-NOV-22,Cash Withdrawal From A C No,Credit
2,2,2298226,1500.0,01-NOV-22,Cash withdraw from Micro Merchant point MM A C,Debit
3,3,4066427,1500.0,01-NOV-22,Credit Against Merchant Cash Withdraw Service ...,Credit
4,4,1978564,2550.0,01-NOV-22,Cash withdraw from Micro Merchant point MM A C,Debit


In [ ]:
data['NARATION'] = data['NARATION'].str.lower()
data['DR_CR'] = data['DR_CR'].str.lower()

In [ ]:
from nltk.corpus import stopwords

# Perform tokenization, stemming, and stop word removal
stop_words = set(stopwords.words('english'))
stop_words |= {'cash', 'a', 'c', 'no'}

data['NARATION'].fillna('', inplace=True)

In [ ]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

In [ ]:
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Define a function to perform lemmatization on a text
def lemmatize_text(text):
    # Tokenize the text into words
    tokens = word_tokenize(text)
    # Lemmatize each word and join them back into a sentence
    lemmatized_text = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return lemmatized_text

# Apply lemmatization to the "NARATION" column
data['Lem_Tokens'] = data['NARATION'].apply(lemmatize_text)
data.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming you have a list of tokenized sentences named 'tokens_list'

# Convert the tokenized sentences into strings
sentences = [' '.join(tokens) for tokens in data['Tokens']]

# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the sentences to obtain the TF-IDF matrix
tfidf_matrix = vectorizer.fit_transform(sentences)

# Get the feature words (vocabulary) from the vectorizer
feature_words = vectorizer.vocabulary_

# Print the feature words
print(feature_words)
print(data.head)

In [ ]:
feature_words_per_row = [list(set(row_tokens) & set(feature_words)) for row_tokens in data['Tokens']]

# data = data.assign(FeatureWords=[list(set(row_tokens) & set(feature_words)) for row_tokens in data['Tokens']])
# print(data.head())

from sklearn.cluster import KMeans

# Define the number of clusters
num_clusters = 5
amounts_array = data['AMOUNT'].to_numpy()

# Create a feature matrix by combining feature words and amounts
data_matrix = np.column_stack((feature_words_per_row, amounts_array))

# Perform K-means clustering
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(data_matrix)

# Get the cluster labels
cluster_labels = kmeans.labels_

# Assign the cluster labels to your original dataframe
data['Cluster'] = cluster_labels
print(data.head())

In [ ]:
# from sklearn.cluster import KMeans

# # Define the number of clusters
# num_clusters = 5
# amounts_array = data['AMOUNT'].to_numpy()

# # Create a feature matrix by combining feature words and amounts
# data_matrix = np.column_stack((feature_words_per_row, amounts_array))

# # Perform K-means clustering
# kmeans = KMeans(n_clusters=num_clusters)
# kmeans.fit(data_matrix)

# # Get the cluster labels
# cluster_labels = kmeans.labels_

# # Assign the cluster labels to your original dataframe
# data['Cluster'] = cluster_labels

In [ ]:
# from sklearn.cluster import MiniBatchKMeans
# from sklearn.feature_extraction.text import CountVectorizer


# vectorizer = CountVectorizer()

# # Fit and transform the documents to obtain the document-term matrix
# X = vectorizer.fit_transform(data['NARATION'])

# # Initialize the MiniBatchKMeans algorithm
# kmeans = MiniBatchKMeans(n_clusters=3, random_state=42)

# # Fit the algorithm on the document-term matrix
# kmeans.fit(X)

# # Get the cluster labels for each document
# labels = kmeans.labels_
 
# vec = TfidfVectorizer(stop_words="english")
# vec.fit(data.NARATION.values)
# keywords = vec.transform(data.NARATION.values)

In [ ]:
# grouped_data2 = data.groupby('AC_ID')

# # Create a new DataFrame with a list of tuples for each distinct 'ac_no'
# grouped_df = pd.DataFrame([(AC_ID, list(zip(group['AMOUNT'], group['DR_CR'], group['TOKENS'])))
#                            for AC_ID, group in grouped_data2],
#                           columns=['AC_ID', 'TRANSACTIONS'])
# grouped_df.head()


In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# # Assuming 'grouped_df' is the DataFrame containing the 'transactions' column with a list of tuples
# # Assuming the column name is 'transactions'
# # def flat(d) -> str:
# #     l = []
# #     for tuple_item in d:
# #         s = ""
# #         for i in tuple_item:
# #             s = s + " " + str(i)
# #         l.append(s)
# #     return l
                        

# # Flatten the 'transactions' column to create a list of strings
# grouped_df['TRANSACTIONS'] = [[str(' '.join(str(element) for element in tuple_element)) for tuple_element in row] for row in grouped_df['TRANSACTIONS']]

In [ ]:
# grouped_df.head()
# Initialize the TfidfVectorizer
# vectorizer = TfidfVectorizer()

# # Compute the TF-IDF features
# tfidf_features = vectorizer.fit_transform(flatten_transactions)
# from sklearn.feature_extraction.text import TfidfVectorizer

# # Assuming you have a dataframe named 'df' with a column 'text' containing the list of strings
# # For example:
# df = pd.DataFrame({'text': [['apple', 'banana', 'orange'], ['grape', 'kiwi']]})
# print(df)

# # Flatten the list of strings within each row
# flattened_text = [word for sublist in df['text'] for word in sublist]
# print(flattened_text)

# # Initialize the TfidfVectorizer
# vectorizer = TfidfVectorizer()

# # Fit and transform the flattened text data to obtain the TF-IDF matrix
# tfidf_matrix = vectorizer.fit_transform(flattened_text)


# # Print the feature words
# print(tfidf_matrix)

In [ ]:
# # Convert the TF-IDF features to a DataFrame
# tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=vectorizer.get_feature_names())

# # Print the TF-IDF DataFrame
# tfidf_df.head()